## OutputParser
- LangChain의 OutputParser는 언어 모델(LLM)의 응답을 개발자가 원하는 형태로 구조화하고 변환해주는 도구입니다. 
- 모델이 생성한 단순한 텍스트 문자열을 JSON, 리스트, 또는 특정 데이터 객체와 같이 다루기 쉬운 형식으로 자동으로 파싱(parsing)해줍니다.

작동 방식:  
1. 형식 지정 (get_format_instructions): 모델에게 원하는 출력 형식을 알려주는 지침을 생성합니다. 이 지침을 프롬프트에 포함하여 모델이 정해진 형식에 맞춰 응답하도록 유도할 수 있습니다.
2. 파싱 (parse): 모델이 생성한 텍스트 응답을 받아 사전에 정의된 데이터 구조로 변환합니다.

공식 문서에서 더 다양한 종류의 OutputParser들을 확인 할 수 있습니다.  
https://docs.langchain.com/oss/python/langchain/structured-output

## Pydantic OutputParser
Pydantic OutputParser란?
- LLM의 출력을 구조화된 데이터 형태로 변환해주는 도구입니다
- LLM이 생성한 텍스트를 Python 객체로 자동 파싱합니다
- 데이터 검증과 타입 안정성을 보장합니다
- JSON 형태의 응답을 Pydantic 모델로 변환하여 일관된 데이터 구조를 제공합니다
- 예시: 사용자 정보를 이름, 나이, 직업, 이메일 등의 구조화된 형태로 추출합니다

LLM 모델 객체 생성하기


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.0)

Pydantic OutputParser 를 사용하지 않는 예시

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
template = """
주어진 텍스트에서 사용자의 정보를 추출하여 구조화된 형태로 변환해주세요.

입력 텍스트: {text}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
)

# 프롬프트 미리보기 (일반적인 회사 내용으로 수정한 예제)
sample_text = "이영희 대리는 1979년생이고, '알파' 전자회사 마케팅팀 소속이며, 20h@email.com을 사용하고, 3년째 근무중입니다."

formatted_prompt = prompt.format(text=sample_text)

print(formatted_prompt)

In [ ]:
chain = prompt | llm

chain.invoke(formatted_prompt).content

이러한 경우처럼 사용자의 정보가 주어졌을때, 정보안에서 추출하고싶은 데이터들이 있다면,   
Pydantic 모델 형태로 정의된 클래스로 형태로 파싱하여 응답을 받을 수 있습니다.

1. Pydantic 모델 정의하기

In [ ]:
from pydantic import BaseModel, Field

class UserInfo(BaseModel):
    """사용자 정보를 담는 Pydantic 모델"""
    name: str = Field(description="사용자의 이름")
    age: int = Field(description="현재나이")
    job: str = Field(description="직업")
    email: str = Field(description="이메일 주소")
    experience: str = Field(description="경력 또는 경험")

# 모델 출력 확인
print("UserInfo 모델 정의 완료")
print("모델 필드:", UserInfo.model_fields)


### `get_format_instructions()` 메서드
- Pydantic 모델의 스키마를 LLM이 이해할 수 있는 명확한 지침 문자열로 변환한다.  
- 변환된 포맷 지침을 기반으로, LLM이 응답을 Pydantic 모델 구조에 맞춰 생성하도록 도와준다.

In [ ]:
from langchain.output_parsers import PydanticOutputParser

# Pydantic OutputParser 생성
parser = PydanticOutputParser(pydantic_object=UserInfo)

# 파서가 어떤 형식을 요구하는지 확인
print(parser.get_format_instructions())


prompt 재정의
- `text` : 군인 정보가 들어있는 text (사용자 입력을 받을 위치)
- `format_instructions` : parser로 형식 지정

In [ ]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의 (format_instructions 포함)
template = """
주어진 텍스트에서 사용자의 정보를 추출하여 구조화된 형태로 변환해주세요.

입력 텍스트: {text}

{format_instructions}
"""

# PromptTemplate 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 프롬프트 미리보기 (일반적인 회사 내용으로 수정한 예제)
sample_text = "이영희 대리는 1990년생이고, '알파' 전자회사 마케팅팀 소속이며, 20h@email.com을 사용하고, 3년째 근무중입니다."

formatted_prompt = prompt.format(text=sample_text)

print(formatted_prompt)

LCEL 체인 만들기
- 체인 구조: PromptTemplate → LLM Model

In [ ]:
# LCEL을 사용한 체인 생성
chain = prompt | llm 

# 테스트 데이터
sample_text = "이영희 대리는 1990년생이고, '알파' 전자회사 마케팅팀 소속이며, 20h@email.com을 사용하고, 3년째 근무중입니다."

# 체인 실행
result = chain.invoke({"text": sample_text})

print(result.content)

Prompt에 작성된 것 처럼 json 형태로 응답이 오는것을 확인 할 수 있다.  
이 응답을 `parser` 를 사용해서 `UserInfo` 객체로 변환해보자

`parser()` : LLM의 응답을 파싱하여 정의된 Pydantic 모델의 객체로 변환하는 함수
   - 입력: LLM의 JSON 형식 응답 문자열
   - 출력: Pydantic 모델의 객체로 변환되는 함수

In [ ]:
parsed_result = parser.parse(result.content);

print(parsed_result)

In [ ]:
print(parsed_result.name) # 객체로 값들을 사용 할 수 있다.

chain으로 한번에 엮어서 사용하기

In [ ]:
chain = prompt | llm | parser

test_text = "이동훈은 25세의 데이터 분석가이며, data@company.com 이메일을 사용하고 3년 경력을 가지고 있습니다."


result = chain.invoke({"text": test_text})

print(result)

print(result.name)